### The notebook is used to download planetscope images edited from notebook with custom requiremnets (Kehan Yang, kyang33@uw.edu)

#### Load packages and set up directories
Planet has updated its API in March 2023, so some of the functions may not be 

In [10]:
import os
import glob
from get_planet import *
from os.path import exists

### Authorize Planet account.
You can copy and paste your planet API from your Planet Account setting.

In [11]:
# If you're following along with this notebook, you can enter your API Key on the following line, and uncomment it:
# os.environ['PLANET_API_KEY']='PLAKf9b6328484534380bcc8c6fa56cd27aa'
# Setup the API Key from the `PL_API_KEY` environment variable
PLANET_API_KEY = os.getenv('PLANET_API_KEY')

#### Get your API Key and run validity check
# This gets your API key and prompts you incase your API key is missing or if there are authentication issues

## Get your API Key
try:
    PLANET_API_KEY = 'PLAK198d7255136b4c28b3cac7446a904121' #remove find_api_key and place your api key like 'api-key'
except Exception as e:
    print("Failed to get Planet Key: Try planet init or install Planet Command line tool")
    sys.exit()

# check if API key is valid 
response = requests.get('https://api.planet.com/compute/ops/orders/v2',auth=(PLANET_API_KEY, ""))
if response.status_code==200:
    print('Setup OK: API key valid')
else:
    print(f'Failed with response code {response.status_code}: reinitialize using planet init')


# Run 'planet init' in terminal; Type in planet account (email address) and password and run this chuck again.

Setup OK: API key valid


### Set up directories

In [12]:
# directory for the geomtry, the format has to be geojson
# dir_geom = './data/geom/'
dir_geom = '/home/etboud/projects/data/geom/'

# dir_root = './data/download/'
dir_root = '/home/etboud/projects/data/download/'

# directory where the images will be downloaded. 
dir_download = dir_root + 'images/'
os.makedirs(dir_download, exist_ok=True)

# directory for the download links and image ids 
dir_order_url =  dir_root + 'links/'
os.makedirs(dir_order_url, exist_ok=True)

# change the flag if search and/or download data are required.
flag_search = True
flag_order = True # if this is set to True, the data will be ordered and count in your plan
flag_download = True

### Start to search and/or download data
If flag_download is set to False, the order will not be placed, and your quote will not be consumed. The total areas will be saved in a CSV file, allowing you to estimate the total size of the areas

In [13]:
# define the searching period; 
years = [2016]
# start and end date of each year
start_date = '-06-01T00:00:00'
end_date = '-06-30T12:00:00'

for yr in years:
    yr = str(yr)
    print(yr)

    if flag_search:
        df_search = pd.DataFrame() # save all image ids
        
        #file to store url -- planet data download links
        file_orders = dir_order_url + yr + '_all_URLs.txt'
        start_time = yr + start_date
        end_time = yr + end_date
        print(file_orders)
        
        overlap = 99 # at least with 99% overlap 
        cloud_pct = 0.05 # no more than 5% cloud cover


        #search for geojson file
        fn = glob.glob(dir_geom + "*geojson")
        ID_shp = [id.split("/")[-1] for id in fn]
        df = pd.DataFrame(data = {
            "file": fn, 
            "index":  [i.split("/")[-1] for i in fn],
            "ID": [id.split("/")[-1].split('.')[0] for id in fn]
            })
        df = df.sort_values("index", ascending = True)

        print(df.head())

    # check whether the order url txt file is exist. if exist, read data; otherwise, creat file.
        idx = 0 
        if exists(file_orders):
            order_urls = pd.read_csv(file_orders)
        else:
            order_urls = pd.DataFrame(columns = ["index","ID_geom", "order_url"])
            


        for irow in df.itertuples():

        # Search id 
            print(irow)
            ID_geom = irow.ID.split(".")[0]+ '_' + yr
            print(ID_geom)

            if ID_geom not in order_urls.ID_geom.to_list():

                print('Searching available images ------- ')
                idlist = ft_iterate(item_type='PSScene', # planet has changed the product item type from 'PSScene4Band' with PSScene
                        asset_type= 'ortho_analytic_4b_sr',
                        geom = read_geom(irow.file),#".json"),
                        start = start_time,
                        end = end_time,
                        cloud_cover = cloud_pct, #cloud cover range 0-1 represting 0-100% so 0.5 means max allowed 50% cloud cover
                        ovp = overlap) #% minimum % overlap 0-100

                idlist['ID_geom'] = ID_geom
                print(idlist.shape)
                idlist.sort_values("date")
                df_search = pd.concat([df_search, idlist])


                # print(irow.file)
                if(flag_order):
                    payload_info = order_payload(Name_download = ID_geom, ID_imgs = idlist.id.values.tolist(), File_geom = irow.file)
                    # print(payload_info)
                    print("Pay order:".format(),ID_geom)


                    order_url = order_now(payload_info) # error response 400  

                    order_urls.loc[idx, "index"] = idx        
                    order_urls.loc[idx, "ID_geom"] = ID_geom
                    order_urls.loc[idx, "order_url"] = order_url
                    print(order_url)
                    order_urls.loc[idx, "NUM"] = idlist.shape[0]  
                    order_urls.loc[idx, "Total area"] = sum(idlist['estimated area'])  


                    # order_urls.append(order_url)  # save all URLs
                    order_urls.to_csv(file_orders, index = None)# save all URLs


            idx = idx + 1
        # save all satellite image id and geom info to one file
        df_search.to_csv(dir_order_url+'all_image_info.csv') 

2016
/home/etboud/projects/data/download/links/2016_all_URLs.txt
                                          file        index   ID
0  /home/etboud/projects/data/geom/Q05.geojson  Q05.geojson  Q05
Pandas(Index=0, file='/home/etboud/projects/data/geom/Q05.geojson', index='Q05.geojson', ID='Q05')
Q05_2016
Searching available images ------- 


KeyError: 'features'

In [ ]:
# read order URL from file_orders
flag_download = True
fn = glob.glob(dir_order_url + '/*_all_URLs.txt')

if flag_download:
    for file_orders in fn:
        order_urls_read = pd.read_csv(file_orders)

        for url in order_urls_read.itertuples():
            print(url.order_url)
            # if poll_for_success(url.order_url):
            if os.path.exists(dir_download + url.ID_geom):
                print("Data have been downloaded".format(), dir_download + url.ID_geom)
            else:
                print("start downloading data to".format(), dir_download + url.ID_geom)
                download_results(url.order_url,folder = dir_download + url.ID_geom)

